In [1]:
%matplotlib inline

import numpy as np
import random
import requests as rq
import sys
import io
import pandas as pd
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop
from collections import Counter
from datetime import datetime
import keras
import keras.callbacks
from keras.callbacks import TensorBoard
%load_ext tensorboard

In [2]:
df = pd.read_csv ('/Users/Elise/Desktop/Smith Year #4!/DIS_academics/NeuralNetworks_DeepLearning/word_embeddings/data/all-the-news-2-1.csv')

MemoryError: 

In [ ]:
df.columns

In [ ]:
df[df['publication']=='Fox News'].groupby('year').count()

In [ ]:
df_fox_news = df[df['publication'] == "Fox News"]
df_fox_news = df_fox_news[df_fox_news['year'] == 2018]
df_fox_news = df_fox_news[['article']]
df_fox_news.to_csv('fox_news.csv', index=False)

In [ ]:
df[df['year'] == 2018].groupby('publication').count()

In [ ]:
df_cnn = df[df['publication'] == "CNN"]
df_cnn = df_cnn[df_cnn['year'] == 2018]
df_cnn = df_cnn[['article']]
df_cnn.to_csv('cnn.csv', index=False)

In [ ]:
df_the_hill = df[df['publication'] == "The Hill"]
df_the_hill = df_the_hill[df_the_hill['year'] == 2018]
df_the_hill = df_the_hill[['article']]
df_the_hill.to_csv('the_hill.csv', index=False)

In [ ]:
df_reuters = df[df['publication'] == "Reuters"]
df_reuters = df_reuters[df_reuters['year'] == 2018]
df_reuters = df_reuters[['article']]
df_reuters.to_csv('reuters.csv', index=False)

In [ ]:
df_nyt = df[df['publication'] == "The New York Times"]
df_nyt = df_nyt[df_nyt['year'] == 2018]
df_nyt = df_nyt[['article']]
df_nyt.to_csv('nyt.csv', index=False)